<a href="https://colab.research.google.com/github/vghu/denselite/blob/main/%D0%91%D0%B0%D0%B7%D0%BE%D0%B2%D1%8B%D0%B9_%D0%B1%D0%BB%D0%BE%D0%BA_%7C_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D1%8B%D0%B9_%D1%81%D0%BB%D0%BE%D0%B9_(Dense)_%7C_%D0%94%D0%97_Lite_%7C_%D0%A3%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

In [18]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential 
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense 
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam 
# Подключение утилит для to_categorical
from tensorflow.keras import utils 
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image 
# Подключение библиотеки для работы с массивами
import numpy as np 
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt 
# Подключение модуля для работы с файлами
import os
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline 

In [19]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_light.zip', None, quiet=True)

'hw_light.zip'

In [20]:
# Распаковываем архив hw_light.zip в папку hw_light
!unzip -q hw_light.zip

replace hw_light/0/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [21]:
# Путь к директории с базой
base_dir = '/content/hw_light'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                    target_size=(img_height, img_width),
                                                    color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        elif patch == '3':
            y_train.append(1)
        else: 
            y_train.append(2)

# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)
print(y_train)

Размер массива x_train (302, 20, 20, 1)
Размер массива y_train (302,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]


In [22]:
# Изменение формата входных картинок с 20х20 на 400х1
x_train = x_train.reshape(x_train.shape[0], 400)
# Вывод новой размерности
print(x_train.shape)
# перевели обьект анализа в линейный вид из квадратного оператором reshape
print(x_train)

(302, 400)
[[253. 255. 255. ... 255. 255. 255.]
 [255. 254. 255. ... 255. 255. 255.]
 [255. 251. 255. ... 255. 255. 255.]
 ...
 [255. 253. 255. ... 251. 255. 255.]
 [255. 253. 253. ... 254. 255. 253.]
 [255. 253. 253. ... 254. 255. 253.]]


In [23]:
# Нормирование входных картинок
# Преобразование x_train в числа с плавающей точкой (тип float)
x_train = x_train.astype('float32')
print(x_train)
# Приведение значений к диапазону от 0 до 1
x_train = x_train / 255 
#производим нормирование чтобы каждый элемент в модели имел значение в пределах от 0 до 1 тип переменной float32
print(x_train)

[[253. 255. 255. ... 255. 255. 255.]
 [255. 254. 255. ... 255. 255. 255.]
 [255. 251. 255. ... 255. 255. 255.]
 ...
 [255. 253. 255. ... 251. 255. 255.]
 [255. 253. 253. ... 254. 255. 253.]
 [255. 253. 253. ... 254. 255. 253.]]
[[0.99215686 1.         1.         ... 1.         1.         1.        ]
 [1.         0.99607843 1.         ... 1.         1.         1.        ]
 [1.         0.9843137  1.         ... 1.         1.         1.        ]
 ...
 [1.         0.99215686 1.         ... 0.9843137  1.         1.        ]
 [1.         0.99215686 0.99215686 ... 0.99607843 1.         0.99215686]
 [1.         0.99215686 0.99215686 ... 0.99607843 1.         0.99215686]]


In [24]:
# Преобразование ответов в формат one_hot_encoding
y_train = utils.to_categorical(y_train, 3)
print(y_train)
# т.к. у нас 3 класса обьектов: треугольник, круг, квадрат, нужно ответ перевести к виду 0,0,0 где позиция числа определеят класс обьекта, маркируя позицию 1-ой
# если запустить 2 раза то будет квадрат 3 на 3, поэтому внимательно проверяем что у нас линейная категоризация

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1.

In [25]:
# Список, содержащий количество нейронов для эксперимента
list_neurons = [10, 100, 5000]
# Список, содержащий функции активации для экспериментов
list_activation = ['relu', 'linear']
# Список, содержащий размер batch_size для экспериментов
list_batch = [10, 100, 1000]
# по сути задачи нужно провести 18 экспериментов соответсвенно, нейроны по функции активации на список бачсайза, получается 18

In [26]:
# Список для сохранения точности сети при заданных параметрах
data_list = []
# Перебор значений в списке с количеством нейронов
for neurons in list_neurons:
    # Перебор значений в списке с функциями активации
    for activation in list_activation:
        # Перебор значений в списке в batch_size
        for batch in list_batch:   
            # Создание сети прямого распространения     
            model = Sequential() 
            # Создание полносвязного слоя с текущим количеством нейронов и активацией
            model.add(Dense(neurons, input_dim=400, activation=activation)) 
            # Создание полносвязного слоя с тремя нейронами для каждого класса и softmax-активацией
            model.add(Dense(3, activation='softmax'))
            # Компиляция модели
            model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"]) 
            # Вывод структуры модели
            print(model.summary()) 
            # Вывод текущих параметров сети
            print(f'Neurons: {neurons}, activation: {activation}, batch_size: {batch}')
            # Обучение модели
            history = model.fit(x_train, y_train, validation_split=0.2, batch_size=batch, epochs=10, verbose=1, shuffle=True)
            # Сохранение параметров и точности сети
            data_list.append(('Neurons:', neurons, 'Activation:', activation, 'Batch: ', batch, 
                              'Validation accuracy:', round(history.history['val_accuracy'][9], 3)))
# собственно сама программа по сути цикл в цикле в цикле, с выводом результата в лист data_list

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                4010      
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 4,043
Trainable params: 4,043
Non-trainable params: 0
_________________________________________________________________
None
Neurons: 10, activation: relu, batch_size: 10
Epoch 1/10
25/25 [==============================] - 5s 12ms/step - loss: 1.0254 - accuracy: 0.4772 - val_loss: 1.4942 - val_accuracy: 0.0656
Epoch 2/10
25/25 [==============================] - 0s 4ms/step - loss: 0.8367 - accuracy: 0.5768 - val_loss: 1.3024 - val_accuracy: 0.0820
Epoch 3/10
25/25 [==============================] - 0s 5ms/step - loss: 0.7408 - accuracy: 0.6515 - val_loss: 1.4966 -

In [27]:
# Вывод результатов экспериментов
for i in data_list:
    print(i)

('Neurons:', 10, 'Activation:', 'relu', 'Batch: ', 10, 'Validation accuracy:', 0.311)
('Neurons:', 10, 'Activation:', 'relu', 'Batch: ', 100, 'Validation accuracy:', 0.0)
('Neurons:', 10, 'Activation:', 'relu', 'Batch: ', 1000, 'Validation accuracy:', 0.148)
('Neurons:', 10, 'Activation:', 'linear', 'Batch: ', 10, 'Validation accuracy:', 0.246)
('Neurons:', 10, 'Activation:', 'linear', 'Batch: ', 100, 'Validation accuracy:', 0.328)
('Neurons:', 10, 'Activation:', 'linear', 'Batch: ', 1000, 'Validation accuracy:', 0.377)
('Neurons:', 100, 'Activation:', 'relu', 'Batch: ', 10, 'Validation accuracy:', 0.508)
('Neurons:', 100, 'Activation:', 'relu', 'Batch: ', 100, 'Validation accuracy:', 0.361)
('Neurons:', 100, 'Activation:', 'relu', 'Batch: ', 1000, 'Validation accuracy:', 0.295)
('Neurons:', 100, 'Activation:', 'linear', 'Batch: ', 10, 'Validation accuracy:', 0.443)
('Neurons:', 100, 'Activation:', 'linear', 'Batch: ', 100, 'Validation accuracy:', 0.328)
('Neurons:', 100, 'Activation:'